## Cartes intéractives

**Carte pour se donner une idée du nombre d'habitant et de la densité.**


In [ ]:
#test d'une carte des densités, interactive 
import folium
from branca.colormap import linear

# Coordonnées et populations des grandes villes de l'Occitanie (Latitude, Longitude, Population)
grandes_villes_occitanie = {
    'Toulouse': (43.6047, 1.4442, 479553),
    'Montpellier': (43.6110, 3.8767, 285121),
    'Nîmes': (43.8374, 4.3601, 151001),
    'Perpignan': (42.6986, 2.8954, 121875),
    'Carcassonne': (43.2130, 2.3491, 47038),
    'Albi': (43.9291, 2.1483, 49236),
    'Tarbes': (43.2328, 0.0716, 40356),
    'Auch': (43.6460, 0.5857, 23853),
}

# Coordonnées du centre de l'Occitanie
centre_occitanie = (43.5912, 1.4466)

# Créer une carte centrée sur l'Occitanie
carte_occitanie = folium.Map(location=centre_occitanie, zoom_start=8, tiles='Stamen Terrain')
folium.Marker(centre_occitanie, popup='Centre de l\'Occitanie', icon=folium.Icon(color='red')).add_to(carte_occitanie)
folium.TileLayer('openstreetmap').add_to(carte_occitanie)  # Ajouter une couche OpenStreetMap en arrière-plan

# Créer une colormap en fonction de la population
colormap = linear.YlOrRd_09.scale(min(grandes_villes_occitanie.values(), key=lambda x: x[2])[2],
                                  max(grandes_villes_occitanie.values(), key=lambda x: x[2])[2])

# Ajouter des marqueurs pour chaque grande ville avec une couleur correspondant à la population
for ville, coordonnees in grandes_villes_occitanie.items():
    if coordonnees[2] > 300000:
        icon = folium.Icon(color='blue', icon='star')
    elif coordonnees[2] > 100000:
        icon = folium.Icon(color='green', icon='cloud')
    else:
        icon = folium.Icon(color='red', icon='info-sign')

    folium.Marker(
        location=coordonnees[:2],
        popup=f"{ville}: {coordonnees[2]} habitants",
        icon=icon
    ).add_to(carte_occitanie)

# Utiliser une échelle log pour le rayon des cercles
for ville, coordonnees in grandes_villes_occitanie.items():
    folium.CircleMarker(
        location=coordonnees[:2],
        radius=coordonnees[2] / 50000,
        popup=f"{ville}: {coordonnees[2]} habitants",
        color=colormap(coordonnees[2]),
        fill=True,
        fill_color=colormap(coordonnees[2]),
    ).add_to(carte_occitanie)

# Personnaliser la légende
colormap.caption = 'Population et densité des grandes villes en Occitanie'
colormap.add_to(carte_occitanie)

# Afficher la carte dans le notebook
carte_occitanie


**En passant la souris sur les différents pointeurs on a le taux moyen de pollution des 5 principaux polluants.** 


In [ ]:
# noinspection PyTypeChecker
import pandas as pd
from ipyleaflet import Map, TileLayer, GeoJSON, Marker
import ipywidgets as widgets
from IPython.display import display

# Charger le fichier CSV dans un DataFrame
chemin_fichier_csv = 'Mesure_mensuelle_Region_Occitanie_Polluants_Principaux.csv'
df = pd.read_csv(chemin_fichier_csv)

# Créer une colonne 'geometry' avec les coordonnées X et Y sous forme de GeoJSON
df['geometry'] = df.apply(lambda row: {"type": "Point", "coordinates": [row['X'], row['Y']]}, axis=1)

# Valeur seuil
SEUIL_DE_VALEUR_ELEVEE = 23.7  # Définissez votre seuil ici

# Créer une carte avec le service WMTS
carte = Map(center=(43.611015, 3.876733), zoom=9)

# Ajouter une couche WMTS à la carte
wmts_url = "https://services.arcgisonline.com/arcgis/rest/services/World_Imagery/MapServer/WMTS"
wmts_layer = TileLayer(url=wmts_url, name="WMTS Layer")
carte.add_layer(wmts_layer)

# Créer une GeoJSON FeatureCollection à partir des données de votre DataFrame
geojson_data = {
    "type": "FeatureCollection",
    "features": []
}

# Marqueurs pour les valeurs élevées
high_value_markers = []

for index, row in df.iterrows():
    feature = {
        "type": "Feature",
        "geometry": row['geometry'],
        "properties": {"nom_dept": row['nom_dept'], "valeur": row['valeur']}
    }
    geojson_data['features'].append(feature)

    # Ajouter un marqueur si la valeur de pollution est élevée
    if row['valeur'] > SEUIL_DE_VALEUR_ELEVEE:
        marker = Marker(location=(row['Y'], row['X']), draggable=False, title=f"Valeur: {row['valeur']}")
        high_value_markers.append(marker)

# Créer une couche GeoJSON pour les zones polluées
geojson_layer = GeoJSON(data=geojson_data, style={'color': 'red', 'opacity': 0.8, 'weight': 1.5})
carte.add_layer(geojson_layer)

# Ajouter les marqueurs à la carte
# Ajouter les marqueurs à la carte
for marker in high_value_markers:
    carte.add_layer(marker)



# Afficher la carte 
display(widgets.HBox([carte]))

**Carte interactive des mesures mensuelles de la région Occitanie des principaux polluants en microgramme par mètre cube**


## Résultats moyenne, écart type, valeur max 


In [ ]:
# Calculer la moyenne et l'écart type des valeurs de pollution
moyenne_pollution = df['valeur'].mean()
ecart_type_pollution = df['valeur'].std()

# Définir le seuil comme la moyenne plus 2 fois l'écart type
seuil_valeur_elevee = moyenne_pollution + 2 * ecart_type_pollution

print("Moyenne de la pollution:", moyenne_pollution)
print("Écart type de la pollution:", ecart_type_pollution)
print("Valeur la plus élevée:", seuil_valeur_elevee)

## Carte des différents polluants


In [ ]:
#| echo: FALSE

import pandas as pd
import numpy as np
from ipyleaflet import Map, TileLayer, GeoJSON, Marker, AwesomeIcon
import ipywidgets as widgets
from IPython.display import HTML, display

class Markers:
    def __init__(self, date_creation, polluant,valeur, X, Y):
        self.date_creation = date_creation
        self.polluant = polluant
        self.valeur = valeur
        self.X = X 
        self.Y = Y
    
    def add_to_map(self,carte):
        carte.add_layer(Marker(location=(self.Y,self.X), draggable=False, title=f"Valeur: {self.valeur}"))

def switch(row):
    if row["id_poll_ue"] == 7:
        marker = Markers(row["date_debut"],row["id_poll_ue"],row["valeur"],row["X"],row["Y"])
        vector_O3.append(marker)
        
    elif row["id_poll_ue"] == 8:
        marker = Markers(row["date_debut"],row["id_poll_ue"],row["valeur"],row["X"],row["Y"])
        vector_NO2.append(marker)
       
    elif row["id_poll_ue"] == 9:
        marker = Markers(row["date_debut"],row["id_poll_ue"],row["valeur"],row["X"],row["Y"])
        vector_NOX.append(marker)
        
    elif row["id_poll_ue"] == 38:
        marker = Markers(row["date_debut"],row["id_poll_ue"],row["valeur"],row["X"],row["Y"])
        vector_NO.append(marker)
        
    elif row["id_poll_ue"] == 5:
        marker = Markers(row["date_debut"],row["id_poll_ue"],row["valeur"],row["X"],row["Y"])
        vector_PM10.append(marker)
        
    elif row["id_poll_ue"] == 11:
        marker = Markers(row["date_debut"],row["id_poll_ue"],row["valeur"],row["X"],row["Y"])
        vector_H2S.append(marker)
        
    elif row["id_poll_ue"] == 6001:
        marker = Markers(row["date_debut"],row["id_poll_ue"],row["valeur"],row["X"],row["Y"])
        vector_PM25.append(marker)
        
    elif row["id_poll_ue"] == 1:
        marker = Markers(row["date_debut"],row["id_poll_ue"],row["valeur"],row["X"],row["Y"])
        vector_SO2.append(marker)
        
    else:
        print(error)


# Créer une carte avec le service WMTS




# Initialisation des marqueurs pour les différents polluants
carte_O3 = Map(center=(43.611015, 3.876733), zoom=9)
carte_NO = Map(center=(43.611015, 3.876733), zoom=9)
carte_NO2 = Map(center=(43.611015, 3.876733), zoom=9)
carte_NOX = Map(center=(43.611015, 3.876733), zoom=9)
carte_H2S = Map(center=(43.611015, 3.876733), zoom=9)
carte_PM10 = Map(center=(43.611015, 3.876733), zoom=9)
carte_PM25 = Map(center=(43.611015, 3.876733), zoom=9)
carte_SO2 = Map(center=(43.611015, 3.876733), zoom=9)

vector_O3 = []
vector_NO = []
vector_NO2 = []
vector_NOX = []
vector_H2S = []
vector_PM10 = []
vector_PM25 = []
vector_SO2 = []

# Charger le fichier CSV dans un DataFrame
chemin_fichier_csv = 'Mesure_mensuelle_Region_Occitanie_Polluants_Principaux.csv'
df = pd.read_csv(chemin_fichier_csv)

# Créer une colonne 'geometry' avec les coordonnées X et Y sous forme de GeoJSON
df['geometry'] = df.apply(lambda row: {"type": "Point", "coordinates": [row['X'], row['Y']]}, axis=1)

for index, row in df.iterrows():


    switch(row)


vecteur_cartes=[vector_O3,vector_NO,vector_NO2,vector_NOX,vector_H2S,vector_PM10,vector_PM25,vector_SO2]

v_cartes=[carte_O3,carte_NO,carte_NO2,carte_NOX,carte_H2S,carte_PM10,carte_PM25,carte_SO2]

values_by_location = {}
marker_by_location = []
for vector in vecteur_cartes:
    for mark in vector:
        X, Y = mark.X, mark.Y
        val = mark.valeur
        
        lieu_key = (X, Y)
        if lieu_key in values_by_location:
            values_by_location[lieu_key].append(val)
        else:
            values_by_location[lieu_key] = [val]
    mean_by_location = {lieu: round(np.mean(val), 2) for lieu, val in values_by_location.items()}
    values_by_location = {}
    marker_by_location.append(mean_by_location)
    marker_by_location[0]

wmts_url = "https://services.arcgisonline.com/arcgis/rest/services/World_Imagery/MapServer/WMTS"
wmts_layer = TileLayer(url=wmts_url, name="WMTS Layer") 

maps_box = widgets.VBox()
i=0
for cartes in v_cartes:
    cartes.add_layer(wmts_layer)

    for lieu, moyenne in marker_by_location[i].items():
        marker = Marker(
            location=(lieu[1], lieu[0]),
            draggable=False,
            title=f"Valeur: {moyenne}"
        )
        cartes.add_layer(marker)

    # Add the map to the horizontal box
    maps_box.children += (cartes,)
    i=i+1

display(maps_box)